In [19]:
import xmltodict
import requests
import json
import bs4

In [20]:
with open("config.json") as f:
    config = json.load(f)

In [21]:
with open("payload_data.json") as fp:
    headers = json.load(fp)

In [22]:
session = None
profile = None
def authenticate():
    global session
    global profile
    
    data =  {
        "username": config['user'],
        "password": config['passw'],
        "client": "mRIC"
    }

    auth_requests = requests.post(  "https://mlog.unitymedia.de/getprofiles/ajax-getprofiles", 
                                    data=data, 
                                    headers=headers['headersAuthentication'])

    text_dict = xmltodict.parse(auth_requests.text)
    session = text_dict['profiles']['session']
    profile =   text_dict['profiles']['profile'][-1] \
                if type(text_dict['profiles']['profile']) == list \
                else text_dict['profiles']['profile']

In [23]:
authenticate()
session, profile

('5c77a5b3-9b5b-4dc7-972b-98fffa61e292', 'Innendienst Disponent (FS)')

In [24]:
def generate_records(order_dict, data):

    for row in data:
        idx = 0
        # print(len(row))
        for k, col in enumerate(order_dict):
            if idx >= len(row):
                # print("FIRST IF:", col, idx, k)
                order_dict[col].append('')
            elif int(row[idx]['colid']) == k+1:
                # print("SeCOND IF:", col, idx, k)
                order_dict[col].append(row[idx]['value'])
                idx += 1
            elif int(row[idx]['colid']) > k+1:
                # print("THIRD IF:", col, idx, k)
                order_dict[col].append('')

In [25]:
params = {
    'timeout': '60000',
}

data = f'<?xml version="1.0" encoding="utf-8" standalone="yes"?>\n<message xmlns="http://www.logobject.ch/schema-ns/jmsoverxml">\n  <meta>\n    <type>bindEMailTemplateRequest</type>\n    <expectResponse>true</expectResponse>\n  </meta>\n  <properties>\n    <property type="string" name="username" value="Dobre.Vanessa" />\n    <property type="string" name="token" value="{session}" />\n    <property type="string" name="userprofile" value="{profile}" />\n    <property type="string" name="txtimeout" value="60" />\n  </properties>\n  <body>\n    <entity class="ch.logobject.mlogistics.bl.msg.BindEMailTemplateRequest" xmlns="">\n      <field name="bodyText" value="Hallo Vodafone-Kund:in, leider haben wir Dich nicht erreicht. Ruf uns bitte an, um Deinen Termin mit dem Technik-Service zu vereinbaren: $RegionContact.Contact(CONTACT).contactInformation.phone$ Vielen Dank! Freundliche Grüße, Dein Vodafone-Team" />\n      <field name="emailTemplateId" value="44" />\n      <field name="taskId" value="9398056" />\n    </entity>\n  </body>\n</message>'.encode()
print(data)
response = requests.post(
    'https://mlog.unitymedia.de/jmsbridge/jms/bindEMailTemplateRequest',
    params=params,
    headers=headers['headers'],
    data=data,
)

b'<?xml version="1.0" encoding="utf-8" standalone="yes"?>\n<message xmlns="http://www.logobject.ch/schema-ns/jmsoverxml">\n  <meta>\n    <type>bindEMailTemplateRequest</type>\n    <expectResponse>true</expectResponse>\n  </meta>\n  <properties>\n    <property type="string" name="username" value="Dobre.Vanessa" />\n    <property type="string" name="token" value="5c77a5b3-9b5b-4dc7-972b-98fffa61e292" />\n    <property type="string" name="userprofile" value="Innendienst Disponent (FS)" />\n    <property type="string" name="txtimeout" value="60" />\n  </properties>\n  <body>\n    <entity class="ch.logobject.mlogistics.bl.msg.BindEMailTemplateRequest" xmlns="">\n      <field name="bodyText" value="Hallo Vodafone-Kund:in, leider haben wir Dich nicht erreicht. Ruf uns bitte an, um Deinen Termin mit dem Technik-Service zu vereinbaren: $RegionContact.Contact(CONTACT).contactInformation.phone$ Vielen Dank! Freundliche Gr\xc3\xbc\xc3\x9fe, Dein Vodafone-Team" />\n      <field name="emailTemplateI

In [26]:
response.text

'<?xml version="1.0" encoding="UTF-8"?>\r\n<message xmlns="http://www.logobject.ch/schema-ns/jmsoverxml" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.logobject.ch/schema-ns/jmsoverxml jmsoverxml.xsd">\r\n  <meta>\r\n    <id>52f387a6-fdfc-4804-9895-919bd5fbb5bd</id>\r\n    <correlationid>445ca9f4-675f-4eb0-a95d-7751b10336d7</correlationid>\r\n    <type>bindEMailTemplateResponse</type>\r\n    <expiration>0</expiration>\r\n    <priority>4</priority>\r\n    <redelivered>false</redelivered>\r\n    <timestamp>1707329419243</timestamp>\r\n    <expectResponse>false</expectResponse>\r\n  </meta>\r\n  <properties />\r\n  <body>\r\n    <entity xmlns="" class="ch.logobject.mlogistics.bl.msg.BindEMailTemplateResponse">\r\n      <field name="bodyText" value="Hallo Vodafone-Kund:in, leider haben wir Dich nicht erreicht. Ruf uns bitte an, um Deinen Termin mit dem Technik-Service zu vereinbaren: 0771 8006460 Vielen Dank! Freundliche GrÃ¼Ã\x9fe, Dein Vodafone-Team

In [27]:
first_response_soup = bs4.BeautifulSoup(response.text, features="xml")

In [28]:
first_response_soup

<?xml version="1.0" encoding="utf-8"?>
<message xmlns="http://www.logobject.ch/schema-ns/jmsoverxml" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.logobject.ch/schema-ns/jmsoverxml jmsoverxml.xsd">
<meta>
<id>52f387a6-fdfc-4804-9895-919bd5fbb5bd</id>
<correlationid>445ca9f4-675f-4eb0-a95d-7751b10336d7</correlationid>
<type>bindEMailTemplateResponse</type>
<expiration>0</expiration>
<priority>4</priority>
<redelivered>false</redelivered>
<timestamp>1707329419243</timestamp>
<expectResponse>false</expectResponse>
</meta>
<properties/>
<body>
<entity class="ch.logobject.mlogistics.bl.msg.BindEMailTemplateResponse" xmlns="">
<field name="bodyText" value="Hallo Vodafone-Kund:in, leider haben wir Dich nicht erreicht. Ruf uns bitte an, um Deinen Termin mit dem Technik-Service zu vereinbaren: 0771 8006460 Vielen Dank! Freundliche GrÃ¼Ãe, Dein Vodafone-Team"/>
<field name="emailTemplateId" value="44"/>
<field name="html" value="false"/>
<field name="subje

In [37]:
params = {
    'timeout': '60000',
}

data = f'<?xml version="1.0" encoding="utf-8" standalone="yes"?>\n<message xmlns="http://www.logobject.ch/schema-ns/jmsoverxml">\n  <meta>\n    <type>sendSmsRequest</type>\n    <expectResponse>true</expectResponse>\n  </meta>\n  <properties>\n    <property type="string" name="username" value="Dobre.Vanessa" />\n    <property type="string" name="token" value="{session}" />\n    <property type="string" name="userprofile" value="{profile}" />\n    <property type="string" name="txtimeout" value="60" />\n  </properties>\n  <body>\n    <entity class="ch.logobject.mlogistics.bl.msg.SendSmsRequest" xmlns="">\n      <field name="emailTemplate" value="SMS_KUNDE_RÜCKRUFBITTE" />\n      <field name="messageText" value="Hallo Vodafone-Kund:in, leider haben wir Dich nicht erreicht. Ruf uns bitte an, um Deinen Termin mit dem Technik-Service zu vereinbaren: 0771 8006460 Vielen Dank! Freundliche Grüße, Dein Vodafone-Team" />\n      <field name="messageType" value="task" />\n      <field name="mobileNumbers" value="{{\'+491741394766\'}}" />\n      <field name="taskId" value="9398056" />\n    </entity>\n  </body>\n</message>'.encode()

response = requests.post(
    'https://mlog.unitymedia.de/jmsbridge/jms/sendSmsRequest',
    params=params,
    headers=headers['headers'],
    data=data,
)

In [38]:
response.content

b'<?xml version="1.0" encoding="UTF-8"?>\r\n<message xmlns="http://www.logobject.ch/schema-ns/jmsoverxml" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.logobject.ch/schema-ns/jmsoverxml jmsoverxml.xsd">\r\n  <meta>\r\n    <id>5022a793-19ad-4479-a2c5-0492b02db07f</id>\r\n    <correlationid>c6d75c64-7c2b-4c4a-b72f-3136096d6c26</correlationid>\r\n    <type>ack</type>\r\n    <expiration>0</expiration>\r\n    <priority>4</priority>\r\n    <redelivered>false</redelivered>\r\n    <timestamp>1707329593441</timestamp>\r\n    <expectResponse>false</expectResponse>\r\n  </meta>\r\n  <properties />\r\n  <body />\r\n</message>\r\n\r\n'

In [39]:
second_response_soup = bs4.BeautifulSoup(response.text, features="xml")

In [40]:
second_response_soup

<?xml version="1.0" encoding="utf-8"?>
<message xmlns="http://www.logobject.ch/schema-ns/jmsoverxml" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.logobject.ch/schema-ns/jmsoverxml jmsoverxml.xsd">
<meta>
<id>5022a793-19ad-4479-a2c5-0492b02db07f</id>
<correlationid>c6d75c64-7c2b-4c4a-b72f-3136096d6c26</correlationid>
<type>ack</type>
<expiration>0</expiration>
<priority>4</priority>
<redelivered>false</redelivered>
<timestamp>1707329593441</timestamp>
<expectResponse>false</expectResponse>
</meta>
<properties/>
<body/>
</message>